# Estimating biomarker ordering

>The sampler for the biomarker ordering can be a bit tricker. The simplest way to do it might be to do a Metropolis-Hastings step where you select two indicies and propose swapping their order. Then you can work out the relative probabilities, evaluate and then accept/reject based on that. It's not the fastest sampler, but it is a lot more straightforward than some ways of doing it.  

In the following, we assume we know the actual $\theta$ and $\phi$ values. Other than those, we do not know anything except for participants' observed biomarker values. And we want to estimate the current order in which different biomarkers are affected by the disease in question. 

In [1]:
import pandas as pd 
import numpy as np 
import re 
import altair as alt 
import matplotlib.pyplot as plt 
from collections import Counter

We only have three columns: biomarker, participant, and measurement. 

In [2]:
data = pd.read_csv('data/participant_data.csv')
data.Biomarker = [re.sub("Biomarker ", "", text) for text in data.Biomarker.tolist()]
data_we_have = data.drop(['k_j', 'S_n', 'affected_or_not'], axis = 1)
data_we_have.head()

,Biomarker,participant,measurement
0,0,0,1.553771
1,0,1,1.010715
2,0,2,0.585092
3,0,3,0.800070
4,0,4,0.884383


In [3]:
theta_phi = pd.read_csv('data/means_vars.csv')
theta_phi.head()

,biomarker,theta_mean,theta_var,phi_mean,phi_var
0,0,1.0,0.3,32.0,6.3
1,1,3.0,0.5,31.0,7.4
2,2,5.0,0.2,34.0,9.4
3,3,6.0,1.3,36.0,4.9
4,4,8.0,3.3,38.0,2.5


In [4]:
type(theta_phi['biomarker'][0])

numpy.int64

In [5]:
def fill_up_pdata(pdata, k_j):
    '''Fill up participant data using k_j
    Input:
    - pdata: a dataframe of ten biomarker values for a specific participant 
    - k_j: a scalar
    '''
    data = pdata.copy()
    data['k_j'] = k_j
    data['affected'] = data.apply(lambda row: row.k_j >= row.S_n, axis = 1)
    return data 

In [6]:
def compute_single_measurement_likelihood(theta_phi, biomarker, affected, measurement):
    '''Computes the likelihood of the measurement value of a single biomarker
    We know the normal distribution defined by either theta or phi
    and we know the measurement. This will give us the probability
    of the given measurement. 

    input:
    - theta_phi: the dataframe containing theta and phi values for each biomarker
    - biomarker: an integer between 0 and 9 
    - affected: boolean 
    - measurement: the observed value for a biomarker in a specific participant

    output: a number 
    '''
    biomarker_params = theta_phi[theta_phi.biomarker == biomarker].reset_index()
    mu = biomarker_params['theta_mean'][0] if affected else biomarker_params['phi_mean'][0]
    var = biomarker_params['theta_var'][0] if affected else biomarker_params['phi_var'][0]
    sigma = np.sqrt(var)
    return np.exp(-(measurement - mu)**2/(2*sigma**2))/np.sqrt(2*np.pi*sigma**2)

In [7]:
def compute_likelihood(pdata, k_j):
    '''This implementes the formula of https://ebm-book2.vercel.app/distributions.html#known-k-j
    '''
    data = fill_up_pdata(pdata, k_j)
    likelihood = 1
    for i, row in data.iterrows():
        biomarker = int(row['Biomarker'])
        measurement = row['measurement']
        affected = row['affected']
        likelihood *= compute_single_measurement_likelihood(theta_phi, biomarker, affected, measurement)
    return likelihood

## Testing

The above functions can compute the likelihood of a participant's sequence of biomarker data, given that we know the exact ordering and we assume a `k_j`. Next, we will test those functions by selecting a specific participant. We compute the likelihood by trying all possible `k_j` and see whether the one with the highest likelihood is the real `k_j` in the original data. 

In [8]:
p = 15 # we chose this participant
pdata = data[data.participant == p].reset_index(drop=True)
pdata

,Biomarker,participant,measurement,k_j,S_n,affected_or_not
0,0,15,1.105621,8,1,affected
1,1,15,27.486492,8,10,not_affected
2,2,15,4.962781,8,5,affected
3,3,15,5.688180,8,6,affected
4,4,15,13.480340,8,4,affected
5,5,15,0.954148,8,2,affected
6,6,15,30.634635,8,9,not_affected
7,7,15,2.360286,8,3,affected
8,8,15,6.353015,8,8,affected
9,9,15,9.242558,8,7,affected


In [9]:
# ordering of biomarker affected by the disease
real_ordering_dic = dict(zip(np.arange(10), pdata.S_n))
real_ordering_dic

{0: 1, 1: 10, 2: 5, 3: 6, 4: 4, 5: 2, 6: 9, 7: 3, 8: 8, 9: 7}

In [10]:
# get the participant data without k_j, S_n, and affected or not
pdata = data_we_have[data_we_have.participant == p].reset_index(drop=True)
# obtain real ordering:
pdata['S_n'] = pdata.apply(lambda row: real_ordering_dic[int(row['Biomarker'])], axis = 1)
pdata

,Biomarker,participant,measurement,S_n
0,0,15,1.105621,1
1,1,15,27.486492,10
2,2,15,4.962781,5
3,3,15,5.688180,6
4,4,15,13.480340,4
5,5,15,0.954148,2
6,6,15,30.634635,9
7,7,15,2.360286,3
8,8,15,6.353015,8
9,9,15,9.242558,7


In [11]:
num_biomarkers = len(pdata.Biomarker.unique())
# calculate likelihood for all possible k_j
likelihood_list = [compute_likelihood(pdata=pdata, k_j=x) for x in range(num_biomarkers+1)]
kjs = np.arange(11)
dic = dict(zip(kjs, likelihood_list))
df = pd.DataFrame.from_dict(dic, orient='index', columns=['likelihood']).reset_index()
df.sort_values('likelihood', ascending=False)

,index,likelihood
8,8,7.174102e-08
7,7,1.518427e-30
6,6,1.214277e-86
5,5,1.244013e-127
4,4,6.062227e-148
3,3,3.967089e-198
9,9,5.816207e-200
2,2,4.884801e-225
1,1,1.951156e-278
0,0,5.482933e-312


From the above result, we can see that the most likelihood `k_j` is 8, which is in fact the real `k_j` in the participant data. 

## Metropolis-Hastings Algorithm Implementation

Next, we will implement the metropolis-hastings algorithm using the above functions. 

In [12]:
def average_all_likelihood(pdata, num_biomarkers):
    '''This is to compute https://ebm-book2.vercel.app/distributions.html#unknown-k-j
    '''
    return np.mean([compute_likelihood(pdata=pdata, k_j=x) for x in range(num_biomarkers+1)])

In [13]:
def metropolis_hastings(pdata, num_biomarkers, iterations):
    '''Implement the metropolis-hastings algorithm
    Inputs: 
        - pdata: a dataframe of ten biomarker values for a specific participant
        - num_biomarker: the number of unique biomarkers
        - iterations: number of iterations

    Outputs:
        - best_order: a numpy array
        - best_likelihood: a scalar 
    '''
    # initialize an ordering and likelihood
    best_order = np.arange(num_biomarkers)
    best_likelihood = -np.inf 
    for it in range(iterations):
        new_order = best_order.copy()
        # randomly select two indices
        a, b = np.random.choice(num_biomarkers, 2, replace=False)
        # swaping the order
        new_order[a], new_order[b] = new_order[b], new_order[a]
        pdata['S_n'] = new_order
        likelihood = average_all_likelihood(pdata, num_biomarkers)
        if likelihood > best_likelihood:
            best_likelihood = likelihood 
            best_order = new_order
        else: 
            ratio = likelihood/best_likelihood
            random_number = np.random(0,1)
            if random_number < ratio:
                best_likelihood = likelihood
                best_order = new_order
    return best_order, best_likelihood


Let's test the algorithm with one specific participant:

In [14]:
pdata = data_we_have[data_we_have.participant == p].reset_index(drop=True)

In [15]:
num_biomarkers = len(pdata.Biomarker.unique())
num_iterations = 1000
best_order, best_likelihood = metropolis_hastings(pdata, num_biomarkers, num_iterations)
best_order

array([0, 8, 7, 6, 4, 3, 9, 2, 1, 5])

In [16]:
# real order
real_ordering = np.array(list(real_ordering_dic.values()))
real_ordering

array([ 1, 10,  5,  6,  4,  2,  9,  3,  8,  7])

We find that the `best_order` obtained from our algorithm is different from the real ordering. 

### All participants

One problem is that we only considered one specific participant; we have have 100 participants. We can do estimate the ordering for all participants:

In [17]:
num_participants = len(data.participant.unique())
p_order_dict = {}
num_biomarkers = len(pdata.Biomarker.unique())
num_iterations = 10
for p in range(num_participants):
    pdata = data_we_have[data_we_have.participant == p].reset_index(drop=True)
    best_order, best_likelihood = metropolis_hastings(pdata, num_biomarkers, num_iterations)
    p_order_dict[p] = best_order
    # print(f"participant #{p} is done.")

In [18]:
estimated_orderings = list(p_order_dict.values())
tuples = [tuple(arr) for arr in estimated_orderings]
counter = Counter(tuples)
most_common_tuple, count = counter.most_common(1)[0]
most_common_tuple, count

((1, 5, 4, 7, 2, 0, 6, 3, 8, 9), 2)

In [19]:
# to see whether the real ordering is in the estimated ordering list
result = any(np.array_equal(real_ordering, arr) for arr in estimated_orderings)
result

False

### Conclusion

It seems this method is slow and also not accurate. The correct ordering is not in the estimated ordering array. I am not sure whether it is because the iterations (100) is too small. 